In [33]:
import pandas as pd
import numpy as np
from scipy import stats

### Loading Data 

In [34]:
# URL of the Google Drive file
url = 'Fraud Detection - Financial transactions.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(url)

# Display the DataFrame
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [35]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [37]:
df.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [38]:
# Count unique values in each column of the DataFrame
unique_values_count = df.nunique()
unique_values_count

step                  743
type                    5
amount            5316900
nameOrig          6353307
oldbalanceOrg     1845844
newbalanceOrig    2682586
nameDest          2722362
oldbalanceDest    3614697
newbalanceDest    3555499
isFraud                 2
isFlaggedFraud          2
dtype: int64

In [39]:
# Calculate the range of values for each numeric column in the DataFrame
ranges = df.select_dtypes(include=['int64', 'float64']).apply(lambda x: x.max() - x.min())
ranges

step              7.420000e+02
amount            9.244552e+07
oldbalanceOrg     5.958504e+07
newbalanceOrig    4.958504e+07
oldbalanceDest    3.560159e+08
newbalanceDest    3.561793e+08
isFraud           1.000000e+00
isFlaggedFraud    1.000000e+00
dtype: float64

In [40]:
# Check if column names correspond to what those columns store
column_checks = {
    "step": df["step"].dtype == 'int64',
    "type": df["type"].dtype == 'object',
    "amount": df["amount"].dtype == 'float64',
    "nameOrig": df["nameOrig"].dtype == 'object',
    "oldbalanceOrg": df["oldbalanceOrg"].dtype == 'float64',
    "newbalanceOrig": df["newbalanceOrig"].dtype == 'float64',
    "nameDest": df["nameDest"].dtype == 'object',
    "oldbalanceDest": df["oldbalanceDest"].dtype == 'float64',
    "newbalanceDest": df["newbalanceDest"].dtype == 'float64',
    "isFraud": df["isFraud"].dtype == 'int64',
    "isFlaggedFraud": df["isFlaggedFraud"].dtype == 'int64'
}

column_checks

{'step': True,
 'type': True,
 'amount': True,
 'nameOrig': True,
 'oldbalanceOrg': True,
 'newbalanceOrig': True,
 'nameDest': True,
 'oldbalanceDest': True,
 'newbalanceDest': True,
 'isFraud': True,
 'isFlaggedFraud': True}

In [41]:
# Calculate summary statistics for each column
summary_stats = df.describe(include='all').transpose()

# Calculate additional statistics
numeric_df = df.select_dtypes(include=['int64', 'float64'])
summary_stats['range'] = numeric_df.apply(lambda x: x.max() - x.min())
summary_stats['mode'] = df.mode().iloc[0]
summary_stats['median'] = numeric_df.median()
summary_stats['std'] = numeric_df.std()
summary_stats['unique'] = df.nunique()

# Reorder columns for better readability
summary_stats = summary_stats[['mean', 'median', 'mode', 'std', 'range', 'unique', 'count', 'min', 'max']]
summary_stats.rename(columns={'50%': 'median'}, inplace=True)

# Display the summary statistics
summary_stats

,mean,median,mode,std,range,unique,count,min,max
step,243.397246,239.000,19.0,1.423320e+02,7.420000e+02,743,6362620.0,1.0,743.0
type,NaN,NaN,CASH_OUT,NaN,NaN,5,6362620,NaN,NaN
amount,179861.903549,74871.940,10000000.0,6.038582e+05,9.244552e+07,5316900,6362620.0,0.0,92445516.64
nameOrig,NaN,NaN,C1065307291,NaN,NaN,6353307,6362620,NaN,NaN
oldbalanceOrg,833883.104074,14208.000,0.0,2.888243e+06,5.958504e+07,1845844,6362620.0,0.0,59585040.37
newbalanceOrig,855113.668579,0.000,0.0,2.924049e+06,4.958504e+07,2682586,6362620.0,0.0,49585040.37
nameDest,NaN,NaN,C1286084959,NaN,NaN,2722362,6362620,NaN,NaN
oldbalanceDest,1100701.66652,132705.665,0.0,3.399180e+06,3.560159e+08,3614697,6362620.0,0.0,356015889.35
newbalanceDest,1224996.398202,214661.440,0.0,3.674129e+06,3.561793e+08,3555499,6362620.0,0.0,356179278.92
isFraud,0.001291,0.000,0.0,3.590480e-02,1.000000e+00,2,6362620.0,0.0,1.0


### Handle Missing Values

In [42]:
# Check for NA or missing values in the DataFrame
missing_values = df.isna().sum()

# Display the count of missing values for each column
missing_values

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [43]:
# Impute missing values with median for numerical columns
numerical_cols = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
for col in numerical_cols:
    df[col].fillna(df[col].median(), inplace=True)

In [44]:
# For categorical columns (if any), fill with a placeholder or mode
df['type'].fillna('UNKNOWN', inplace=True)

In [45]:
# Check for duplicate rows in the DataFrame
duplicate_rows = df[df.duplicated()]

# Display the duplicate rows
duplicate_rows

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud


### Encode Categorical Variables

In [46]:
# One-hot encode the 'type' column
df = pd.get_dummies(df, columns=['type'], drop_first=True)

# Alternatively, use Label Encoding if preferred
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# df['type'] = le.fit_transform(df['type'])

### Feature Engineering

In [47]:
# Calculate balance change ratio for origin account
df['balance_change_ratio_org'] = (df['newbalanceOrig'] - df['oldbalanceOrg']) / df['oldbalanceOrg']
df['balance_change_ratio_org'] = df['balance_change_ratio_org'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Calculate balance change ratio for destination account
df['balance_change_ratio_dest'] = (df['newbalanceDest'] - df['oldbalanceDest']) / df['oldbalanceDest']
df['balance_change_ratio_dest'] = df['balance_change_ratio_dest'].replace([np.inf, -np.inf], np.nan).fillna(0)

### Normalize Numerical Features

In [48]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_cols = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

### Detect and Handle Outliers

In [49]:
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)]

# Detect outliers in 'amount'
outliers = detect_outliers_iqr(df, 'amount')
print("Outliers in 'amount':", len(outliers))

# Cap outliers instead of removing them
df['amount'] = df['amount'].clip(lower=df['amount'].quantile(0.01), upper=df['amount'].quantile(0.99))

Outliers in 'amount': 338078


### Balance the Dataset

In [51]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

# Separate features and target
X = df.drop(columns=['isFraud', 'isFlaggedFraud'])
y = df['isFraud']

# Convert categorical features to numeric
label_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = le

# Apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Create a new balanced DataFrame
df_balanced = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='isFraud')], axis=1)

### Save the Cleaned Dataset

In [52]:
# Save the cleaned dataset
df_balanced.to_csv("cleaned_financial_transactions.csv", index=False)